# Agentes

A diferencia de los sistemas de LLMs tradicionales, que solo generan respuestas basadas en su entrenamiento y son más reactivos, o inclusive que pueden llegar a generar respuestas más genéricas, los Agentes son sistemas más autónomos que pueden:

* Razonar y tomar decisiones propias
* Ejecutar acciones concretas - utilizando diferentes herramientas (APIs, búsquedas, cálculos, conexión con otros sistemas)
* Mantener memoria y contexto de interacciones previas

Mientras un LLM tradicional solo te diría qué hacer, un Agente puede entender, planificar y ejecutar acciones por sí mismo para resolver tareas.
Podrá razonar qué herramientas son apropiadas a usar.
La diferencia está en la capacidad de acción, no solo en el conocimiento.

A las acciones de Razonar y Actuar de los agentes les denominaremos "ReAct"


In [ ]:
%pip install -U langchain-google-genai
%pip install -q -U google-generativeai

In [ ]:
%pip install langchain-community python-dotenv langchain-google-genai google-generativeai

In [1]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
#response = model.generate_content("Explain how AI works")
#print(response.text)

/Users/edudom/Documents/agents/agents_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Librería para mandar a llamar las tools a las que puede acceder nuestro agente.

Lista de herramientas disponibles: https://python.langchain.com/v0.1/docs/integrations/tools/

In [ ]:
%pip install langchain
%pip install -U langchain-community

In [2]:
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor

## Definimos herramientas a las que tendrá acceso el agente

Usaremos SerpApi (https://serpapi.com/), esta api nos ayudará a realizar búsquebas en la web para extraer resultados de google y otros motores de búsqueda

In [ ]:
%pip install --upgrade --quiet  google-search-results

In [4]:
from dotenv import load_dotenv
import os
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

# Cargar variables de entorno del archivo .env
load_dotenv()
api_key = os.getenv("SERPAPI_API_KEY")

tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [5]:
tool.run("Google")

'\nQuery: Google\nstock: GOOG:NASDAQ\nprice: $165.80\npercentage: 0.45\nmovement: Up\nus: price = 42931.6, movement = Down\neurope: price = 19461.19, movement = Down\nasia: price = 38458.89, movement = Down\n'

In [15]:
import os
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

def setup_agent():
    # Cargar variables de entorno
    load_dotenv()
    
    # Verificar API keys
    required_keys = ["GEMINI_API_KEY", "SERPAPI_API_KEY"]
    missing_keys = [key for key in required_keys if not os.getenv(key)]
    
    if missing_keys:
        raise ValueError(f"Faltan las siguientes API keys en el archivo .env: {', '.join(missing_keys)}")

    # Inicializar el modelo de Gemini
    llm = GoogleGenerativeAI(
        model="gemini-pro",
        google_api_key=os.getenv("GEMINI_API_KEY"),
        temperature=0.7
    )
    
    # Configurar SerpAPI para búsquedas
    search = SerpAPIWrapper()
    
    # Crear herramientas personalizadas
    tools = [
        Tool(
            name="google-finance",
            func=search.run,
            description="Útil para cuando necesites buscar información actualizada sobre acciones, empresas o cualquier otro tema"
        )
    ]
    
    # Inicializar el agente
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
        verbose=True
    )
    
    return agent

if __name__ == "__main__":
    try:
        agent = setup_agent()
        response = agent.run("""
        Realiza un análisis completo de Google  como inversión potencial. 
        Considera los siguientes aspectos:
        1. Rendimiento actual de la acción y tendencia de los últimos 6 meses
        2. Principales métricas financieras y crecimiento
        3. Noticias recientes que puedan afectar el valor de la empresa
        4. Comparación con competidores principales
        5. Factores de riesgo actuales
        
        Basado en estos datos, proporciona una recomendación sobre si sería un buen momento para invertir o no""")
        print(response)
    except Exception as e:
        print(f"Error: {str(e)}")



> Entering new AgentExecutor chain...
I need to look up the stock performance of Google, its financial metrics, recent news, and compare it with its competitors to assess its investment potential.
Action: google-finance
Action Input: google-finance(query="Google stock performance")
Observation: ['GOOGLEFINANCE Fetches current or historical securities information from Google Finance. Sample Usage GOOGLEFINANCE("NASDAQ:GOOG", "price", DATE(2014,1,1), DATE ...', 'You can access all of the Google Finance data in Google Sheets. This is helpful if you want to download, analyze, and visualize the data.', 'The GOOGLEFINANCE function allows you to import real-time financial and currency market data directly into Google Sheets, tracking both current ...', 'The GOOGLEFINANCE function allows you to import real-time financial and currency market data straight into Google Sheets.', 'GOOGLEFINANCE is a Google Sheets function that tracks real-time financial market trends, stock updates, and currency